<a href="https://colab.research.google.com/github/ggsmith842/AIML-tutorials/blob/main/cohere/Cohere_AI_Agents.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [14]:
!pip install cohere -Uq

In [15]:
import cohere
import json
from google.colab import userdata
from typing import List, Dict

In [16]:
co = cohere.ClientV2(api_key=userdata.get("COHERE_API"))

# Creating Tools



In [17]:
def search_faqs(query: str) -> List[Dict]:
  """
  Given a query, returns a list of dictionaries.
  """
  faqs = [
      {
          "text":"Reimbursing Travel Expenses: Easily manage your travel expenses by submitting them through our finance tool. Approvals are prompt and straightforward."
      },
      {
        "text": "Working from Abroad: Working remotely from another country is possible. Simply coordinate with your manager and ensure your availability during core hours."
      }
  ]
  return faqs

In [18]:
def search_emails(query: str) -> List[Dict]:
    emails = [
        {
            "from": "it@co1t.com",
            "to": "david@co1t.com",
            "date": "2024-06-24",
            "subject": "Set Up Your IT",
            "text": "Greetings! To ensure a seamless start, please refer to the attached comprehensive guide, which will assist you in setting up all your work accounts.",
        },
        {
            "from": "john@co1t.com",
            "to": "david@co1t.com",
            "date": "2024-06-24",
            "subject": "First Week Check-In",
            "text": "Hello! I hope you're settling in well. Let's connect briefly tomorrow to discuss how your first week has been going. Also, make sure to join us for a welcoming lunch this Thursday at noon—it's a great opportunity to get to know your colleagues!",
        },
    ]
    return emails

In [19]:
def create_calendar_event(date: str, time: str, duration: int) -> Dict:
  return {
      "is_success": True,
      "message": f"Create a {duration} hour long event at {time} on {date}"
  }

functions_map = {
    "search_faqs": search_faqs,
    "search_emails": search_emails,
    "create_calendar_event": create_calendar_event
}

In [20]:
# Define the tools
tools = [
    {
        "type": "function",
        "function": {
            "name": "search_faqs",
            "description": "Given a user query, searches a company's frequently asked questions (FAQs) list and returns the most relevant matches to the query.",
            "parameters": {
                "type": "object",
                "properties": {
                    "query": {
                        "type": "string",
                        "description": "The query from the user",
                    }
                },
                "required": ["query"],
            },
        },
    },
    {
        "type": "function",
        "function": {
            "name": "search_emails",
            "description": "Given a user query, searches a person's emails and returns the most relevant matches to the query.",
            "parameters": {
                "type": "object",
                "properties": {
                    "query": {
                        "type": "string",
                        "description": "The query from the user",
                    }
                },
                "required": ["query"],
            },
        },
    },
    {
        "type": "function",
        "function": {
            "name": "create_calendar_event",
            "description": "Creates a new calendar event of the specified duration at the specified time and date. A new event cannot be created on the same time as an existing event.",
            "parameters": {
                "type": "object",
                "properties": {
                    "date": {
                        "type": "string",
                        "description": "the date on which the event starts, formatted as mm/dd/yy",
                    },
                    "time": {
                        "type": "string",
                        "description": "the time of the event, formatted using 24h military time formatting",
                    },
                    "duration": {
                        "type": "number",
                        "description": "the number of hours the event lasts for",
                    },
                },
                "required": ["date", "time", "duration"],
            },
        },
    },
]


# Tool-use Workflow

In [21]:
# set system message
system_msg = """
  ## Task and Context
  You are an assistant who helps new employees of Colt with their first week.
  """

# Get user message
message = "Are there any messages about getting setup with IT?"

messages = [
    {"role": "system", "content": system_msg},
    {"role": "user", "content": message}
]

# tool planning and calling
resp = co.chat(
    model="command-a-03-2025", messages=messages, tools=tools
)

In [22]:
if resp.message.tool_calls:
  print("Tool Plan:")
  print(resp.message.tool_plan, "\n")
  print("Tool calls:")
  for tc in resp.message.tool_calls:
    print(
        f"Tool name: {tc.function.name} | Parameters: {tc.function.arguments}"
    )

  messages.append(
      {
          "role": "assistant",
          "tool_calls": resp.message.tool_calls,
          "tool_plan": resp.message.tool_plan
      }
  )

Tool Plan:
I'll search the FAQs for 'getting setup with IT'. 

Tool calls:
Tool name: search_faqs | Parameters: {"query":"getting setup with IT"}


# Tool Execution

In [23]:
for tc in resp.message.tool_calls:
  tool_result = functions_map[tc.function.name](
      **json.loads(tc.function.arguments)
  )
  tool_content = []
  for data in tool_result:
    tool_content.append(
        {
            "type": "document",
            "document": {"data": json.dumps(data)},
        }
    )

  messages.append(
      {
          "role":"tool",
          "tool_call_id": tc.id,
          "content": tool_content
      }

  )

  print("Tool Results:")
  for result in tool_content:
    print(result)

Tool Results:
{'type': 'document', 'document': {'data': '{"text": "Reimbursing Travel Expenses: Easily manage your travel expenses by submitting them through our finance tool. Approvals are prompt and straightforward."}'}}
{'type': 'document', 'document': {'data': '{"text": "Working from Abroad: Working remotely from another country is possible. Simply coordinate with your manager and ensure your availability during core hours."}'}}


# Response and citation generation

In [24]:
resp = co.chat(
    model="command-a-03-2025", messages=messages, tools=tools
)

# Append assistant response to the chat history
messages.append(
    {"role": "assistant", "content": resp.message.content[0].text}
)

In [25]:
# Print final response
print("Response:")
print(resp.message.content[0].text)
print("=" * 50)

# Print citations (if any)
if resp.message.citations:
    print("\nCITATIONS:")
    for citation in resp.message.citations:
        print(citation, "\n")

Response:
I'm afraid I couldn't find any information about getting set up with IT.


# Multi-step Tool Use

Execute tasks that require tool calls to happen in a sequence.

In [28]:
model = "command-a-03-2025"

system_message = """## Task and Context
You are an assistant who assists new employees of Co1t with their first week. You respond to their questions and assist them with their needs. Today is Monday, June 24, 2024"""


def run_assistant(query, messages=None):
    if messages is None:
        messages = []

    if "system" not in {m.get("role") for m in messages}:
        messages.append({"role": "system", "content": system_message})

    # Step 1: get user message
    print(f"Question:\n{query}")
    print("=" * 50)

    messages.append({"role": "user", "content": query})

    # Step 2: Generate tool calls (if any)
    response = co.chat(model=model, messages=messages, tools=tools)

    while response.message.tool_calls:

        print("Tool plan:")
        print(response.message.tool_plan, "\n")
        print("Tool calls:")
        for tc in response.message.tool_calls:
            print(
                f"Tool name: {tc.function.name} | Parameters: {tc.function.arguments}"
            )
        print("=" * 50)

        messages.append(
            {
                "role": "assistant",
                "tool_calls": response.message.tool_calls,
                "tool_plan": response.message.tool_plan,
            }
        )

        # Step 3: Get tool results
        for idx, tc in enumerate(response.message.tool_calls):
            tool_result = functions_map[tc.function.name](
                **json.loads(tc.function.arguments)
            )
            tool_content = []
            for data in tool_result:
                tool_content.append(
                    {
                        "type": "document",
                        "document": {"data": json.dumps(data)},
                    }
                )
                # Optional: add an "id" field in the "document" object, otherwise IDs are auto-generated
            messages.append(
                {
                    "role": "tool",
                    "tool_call_id": tc.id,
                    "content": tool_content,
                }
            )

        # Step 4: Generate response and citations
        response = co.chat(
            model=model, messages=messages, tools=tools
        )

    messages.append(
        {
            "role": "assistant",
            "content": response.message.content[0].text,
        }
    )

    # Print final response
    print("Response:")
    print(response.message.content[0].text)
    print("=" * 50)

    # Print citations (if any)
    if response.message.citations:
        print("\nCITATIONS:")
        for citation in response.message.citations:
            print(citation, "\n")

    return messages


In [29]:
messages = run_assistant(
    "Can you check if there are any lunch invites, and for those days, create a one-hour event on my calendar at 12PM."
)


Question:
Can you check if there are any lunch invites, and for those days, create a one-hour event on my calendar at 12PM.
Tool plan:
I'll search for lunch invites in the user's emails, and then create a one-hour event on their calendar at 12pm for each day that has a lunch invite. 

Tool calls:
Tool name: search_emails | Parameters: {"query":"lunch invites"}
Tool plan:
I've found a lunch invite for Thursday. I'll now create a one-hour event on the user's calendar at 12pm for Thursday 27th June 2024. 

Tool calls:
Tool name: create_calendar_event | Parameters: {"date":"06/27/24","time":"12:00","duration":1.0}
Response:
I've found a lunch invite for Thursday and created a one-hour event on your calendar at 12pm for that day.

CITATIONS:
start=30 end=38 text='Thursday' sources=[ToolSource(type='tool', id='search_emails_xe00kjt9xhb5:1', tool_output={'date': '2024-06-24', 'from': 'john@co1t.com', 'subject': 'First Week Check-In', 'text': "Hello! I hope you're settling in well. Let's conne